In [1]:
import numpy as np, healpy as hp, prjlib, curvedsky, misctools
from matplotlib.pyplot import *

KeyboardInterrupt: 

In [ ]:
ascale = 1. # 1deg apodization to Galactic mask
pL  = prjlib.init_analysis(wtype='Lmask',ascale=ascale)
pG  = prjlib.init_analysis(wtype='G60',ascale=ascale)
pGL = prjlib.init_analysis(wtype='G60Lmask',ascale=ascale)
nside = 2048

CMB mask

In [ ]:
# G60 x Lmask
if not misctools.check_path(pGL.fmask):
    G60   = hp.fitsfunc.read_map(pGL.fmask_Gfsky,field=2,verbose=False)
    Lmask = hp.fitsfunc.read_map(pL.fmask,verbose=False)
    hp.fitsfunc.write_map(pGL.fmask,G60*Lmask,overwrite=True)
    hp.mollview(G60*Lmask)

Apodization

In [ ]:
for wtype, gal in [('Lmask',3),('G60',2),('G60Lmask',2)]: #index is G70 or G60
    p = prjlib.init_analysis(wtype=wtype,ascale=ascale)
    if not misctools.check_path(p.famask,overwrite=True):
        if wtype in ['G60']:
            amask = 1.
        else:
            mask = hp.fitsfunc.read_map(p.fmask,verbose=False)
            amask = curvedsky.utils.apodize(nside,mask,.5) # apodize original mask
        Gmask = hp.fitsfunc.read_map(p.fmask_Gfsky,field=gal,verbose=False) 
        Amask = curvedsky.utils.apodize(nside,Gmask,ascale) # apodize Galactic mask
        hp.fitsfunc.write_map(p.famask,amask*Amask,overwrite=True)

In [ ]:
for wtype in ['Lmask','G60','G60Lmask']:
#for wtype in ['LmaskN18']:
    p = prjlib.init_analysis(wtype=wtype,ascale=ascale)
    mask = hp.fitsfunc.read_map(p.fmask)
    print('effective sky area',np.mean(mask))
    amask = hp.fitsfunc.read_map(p.famask,verbose=False)
    print('effective sky area',np.mean(amask))
    hp.mollview(mask)
    hp.mollview(amask)

For no-SZ

In [ ]:
ascale = 1.
nside = 2048
pL3  = prjlib.init_analysis(wtype='LmaskDR3',ascale=ascale)
Lmask = hp.fitsfunc.read_map(pL3.fmask)

In [ ]:
amask = curvedsky.utils.apodize(2048,Lmask,.5)
Gmask = hp.fitsfunc.read_map(pL3.fmask_Gfsky,field=3,verbose=False) 
Amask = curvedsky.utils.apodize(nside,Gmask,ascale=ascale) # apodize Galactic mask

In [ ]:
hp.mollview(amask*Amask)

In [ ]:
hp.fitsfunc.write_map(pL3.famask,amask*Amask,overwrite=True)

In [ ]:
amask = hp.fitsfunc.read_map(pL.famask)
hp.mollview(amask)